In [2]:
from openai import OpenAI

api_key_str = "$apikeystr" #replace with a valid key 

client = OpenAI(
    api_key=api_key_str,
    base_url="https://api.moonshot.cn/v1",
)

def chat_completion(prompt_str, given_text):
    completion = client.chat.completions.create(
        model="moonshot-v1-8k",
        messages=[
            {"role": "system", "content": prompt_str},
            {"role": "user", "content": "给定文本：" + given_text}
        ],
        temperature=0,
    )
    return completion.choices[0].message.content

In [6]:
prompt_str = '你是文本分析专家，请理解给定的文本，判断文本的情感类别，可选的情感类别包括：积极和消极。针对输入的多条文本，分析返回其对应的情感类别，输入和输出均以JSON形式表示，输入形式如```[{"id":0,"comment":"菜量太小米饭太多"},{"id":1,"comment":"很快，包装可以，好吃"}]```，输出形式如```[{"id":1,"sentiment":"消极"},{"id":1,"sentiment":"积极"}]```'
given_text = '```[{"id":0,"comment":"超可爱的帅哥，爱了。。。"},{"id":1,"comment":"emo了，花了那么多时间还是一事无成"}]```'

print(chat_completion(prompt_str, given_text))

[{"id":0,"sentiment":"积极"},{"id":1,"sentiment":"消极"}]


In [21]:
import pandas as pd
df = pd.read_csv('./comments-1k.txt',index_col=False,header=0)

In [22]:
df.columns

Index(['label', 'review'], dtype='object')

In [23]:
comments = []
ground_truth = []
for k,v in df.iterrows():
    label = v['label']
    review = v['review']
    if label == 1:
        sentiment = '积极'
    elif label == 0:
        sentiment = '消极'
    comments.append({"id":k,"comment":review})
    ground_truth.append(sentiment)

In [25]:
comments[:5]

[{'id': 0, 'comment': '很快，好吃，味道足，量大'},
 {'id': 1, 'comment': '没有送水没有送水没有送水'},
 {'id': 2, 'comment': '非常快，态度好。'},
 {'id': 3, 'comment': '方便，快捷，味道可口，快递给力'},
 {'id': 4, 'comment': '菜味道很棒！送餐很及时！'}]

In [26]:
ground_truth[:5]

['积极', '积极', '积极', '积极', '积极']

In [46]:
import json
moonshot_results = []
prompt_str = '你是文本分析专家，请理解给定的批量文本，判断每条文本的情感类别，可选的情感类别包括：积极和消极。针对输入的多条文本，分析返回其对应的情感类别，输入和输出必须以严格的JSON字符串表示，不能添加多余字符，输入形式：```[{"id":0,"comment":"菜量太小米饭太多"},{"id":1,"comment":"很快，包装可以，好吃"}]```，输出形式：```[{"id":0,"sentiment":"消极"},{"id":1,"sentiment":"积极"}]```。'
for i in range(0, int(len(comments)/10)):
    #if i>1:
    #    break
    print(sidx,eidx)
    sidx = i*10
    eidx = (i+1)*10
    if eidx>len(comments):
        eidx=len(comments)
    given_text = '```'+json.dumps(comments[sidx:eidx],ensure_ascii=False)+'```'
    print(given_text)
    results = chat_completion(prompt_str, given_text)
    print(results)
    moonshot_results.extend(json.loads(results)) 

360 370
```[{"id": 360, "comment": "菜很好吃，就是炒饭这次给的有点少。"}, {"id": 361, "comment": "快递小哥服务特别好，餐送到还热乎乎的，正好吃。菜品不错。"}, {"id": 362, "comment": "饭很少,要知道就多定两碗了,送餐师傅很棒"}, {"id": 363, "comment": "土豆炖得特别好,适合老人吃,谢谢"}, {"id": 364, "comment": "速度比预计快很多，菜品不错"}, {"id": 365, "comment": "辣子鸡越来越少了。,！！！！！"}, {"id": 366, "comment": "送餐小哥真快，菜也很好吃"}, {"id": 367, "comment": "好吃，好吃，江山城的烤鸭真是挺好吃的"}, {"id": 368, "comment": "挺好的，挺不错的，味道不错"}, {"id": 369, "comment": "菜有点辣，菜量挺好，量大。味道挺好。"}]```
[{"id": 360, "sentiment": "积极"}, {"id": 361, "sentiment": "积极"}, {"id": 362, "sentiment": "积极"}, {"id": 363, "sentiment": "积极"}, {"id": 364, "sentiment": "积极"}, {"id": 365, "sentiment": "消极"}, {"id": 366, "sentiment": "积极"}, {"id": 367, "sentiment": "积极"}, {"id": 368, "sentiment": "积极"}, {"id": 369, "sentiment": "积极"}]
360 370
```[{"id": 370, "comment": "态度很好，效率很高，给十分赞"}, {"id": 371, "comment": "每次量都很足哈！超级赞，好吃"}, {"id": 372, "comment": "送餐很快，可是为啥没餐具呢，用手吃？幸好送餐师傅给了两双百度餐具。"}, {"id": 373, "comment": "百度骑士速度好快啊！而且服务态度超级好～～江山城的饭一直很好吃，走

In [48]:
moonshot_results_bak = moonshot_results

In [49]:
with open("moonshot_result.json","w",encoding="utf-8") as fw:
    json.dump(moonshot_results,fw,ensure_ascii=False,indent=2)

In [51]:
sentiment_dict = {}
for record in moonshot_results:
    sid = record['id']
    sentiment = record['sentiment']
    sentiment_dict[sid] = sentiment

In [64]:
moonshot_prediction_y = []
ground_truth_y = []
for i in range(1000):
    if sentiment_dict[i].strip() == "积极":
        moonshot_prediction_y.append(1)
    else:
        moonshot_prediction_y.append(0)
    if ground_truth[i].strip() == "积极":
        ground_truth_y.append(1)
    else:
        ground_truth_y.append(0)    

In [62]:
# for i in range(1000):
    # print(ground_truth[i],moonshot_prediction[i])


In [65]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
precision = precision_score(ground_truth_y, moonshot_prediction_y, average='binary')
print('precision:', precision)

recall = recall_score(ground_truth_y, moonshot_prediction_y, average='binary')
print('recall:', recall)

f1_score = f1_score(ground_truth_y, moonshot_prediction_y, average='binary')
print('f1_score:', f1_score)

precision: 0.9788583509513742
recall: 0.926
f1_score: 0.9516957862281604
